In [16]:
import pylab as py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [154]:
medical_data = pd.read_csv("dataset/other_train.csv")
personal_data = pd.read_csv("dataset/personal_train.csv")

V predchádzajúcej časti sme v oboch súboroch idetifikovali dva nepomenované stĺpce, ktoré pre nás nemajú žiaden význam,
takže ich teraz môžeme z oboch datasetov odstrániť.

In [155]:
personal_data = personal_data.drop(personal_data.columns[0], axis=1)
medical_data = medical_data.drop(medical_data.columns[0], axis=1)

Dáta spojíme do jednej tabuľky, keďže jedno pozorovanie musí byť opísané len jedným riadkom.

In [265]:
data = pd.merge(personal_data, medical_data, on = ["name", "address"], how = "outer")
data_backup = data

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3982
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              3983 non-null   object 
 1   address           3983 non-null   object 
 2   age               3983 non-null   int64  
 3   sex               3983 non-null   object 
 4   date_of_birth     3983 non-null   object 
 5   kurtosis_oxygen   3974 non-null   float64
 6   pregnant          3971 non-null   object 
 7   education-num     3579 non-null   float64
 8   relationship      3961 non-null   object 
 9   skewness_glucose  3965 non-null   float64
 10  mean_glucose      3963 non-null   float64
 11  std_oxygen        3967 non-null   float64
 12  capital-gain      3969 non-null   float64
 13  skewness_oxygen   3975 non-null   float64
 14  kurtosis_glucose  3968 non-null   float64
 15  personal_info     3974 non-null   object 
 16  education         3971 non-null   object 


Ako prvé môžeme premenovať stĺpce class a fnlwgt, aby ich názvy boli viacej informatívne.

In [ ]:
data = data.rename(columns = {'fnlwgt': 'final_weight', 'class': 'diabetes_presence'}, inplace = False)

### Úprava chybných atribútov

V tejto časti opravíme chyby, ktoré sme idetifikovali v predchádzajúcej časti projektu, prípadne aj novo-objavené chyby,
ak na ne narazíme.


V doleuvedenej tabuľke vidíme chybu v atribúte age. Minimálny vek má hodnotu -1, čo je samozrejme nesprávne. Môžeme to
vyriešiť, tak, že vyrátame nový vek pacienta, keďže máme aj atribút date_of_birth. Toto funguje len za predpokladu, že
date_of_birth nemá nulovú hodnotu a bol zadaný správne. Iným riešením by mohlo byť nahradenie záporného veku
priemerným, alebo najčastejšie sa vyskytujúcim vekom všetkých pacientov.

In [177]:
data["age"].describe()

count    3983.000000
mean       51.844841
std        11.797919
min        -1.000000
25%        44.000000
50%        52.000000
75%        60.000000
max       113.000000
Name: age, dtype: float64

Každý pacient má zadaný vek a dátum narodenia, čiže môžeme vytvoriť funkciu, ktorá vyráta nový vek podľa dátumu.

In [178]:
print(sum(data["age"].isnull()))
print(sum(data["date_of_birth"].isnull()))

0
0


In [179]:
data["date_of_birth"].head(40)

0              1955-08-04
1              1974-05-17
2              1977-01-13
3              28/12/1957
4              1962-07-07
5              1957-03-16
6              1943/07/04
7              1984-05-13
8              1971-05-10
9              1986-11-13
10             1978-11-10
11             1967/07/20
12    1959-01-25 00:00:00
13             1959-12-05
14             1968-10-05
15             1957-06-28
16             1977-04-02
17             1968-05-22
18             1968-05-22
19             1960-12-25
20             30/08/1986
21             1941-07-25
22    1960-05-01 00 00 00
23             1959-01-25
24             1972-10-31
25             1951-06-17
26             1960-11-22
27             1960-09-18
28             1970-04-06
29    1974-09-28 00:00:00
30               77-04-12
31    1950-06-21 00 00 00
32             1955-04-05
33             1955-04-05
34               66-10-30
35             17/08/1964
36             1951-08-18
37             1979-11-29
38          

Ako môžeme vidieť, dátum nemá jednotný formát, čiže bude nutné vytvoriť funkciu, ktorá tento problém odstráni. Vyzerá,
že najpoužívanejší typ formátovania je YYYY-MM-DD, čiže všetky dátumy naň prehodíme.

In [180]:
def unify_date_format(date):

    date = str(date).replace("/", "-")
    date = date[:10]
    date = date.split("-")

    # if date format DD-MM-YYYY
    if len(date[2]) == 4:
        date = date[2] + "-" + date[1] + "-" + date[0]
        return date

    # if date format YY-MM-DD
    elif len(date[0]) == 2 and len(date[2]) == 2:
        date = "19" + date[0] + "-" + date[1] + "-" + date[2]
        return date

    date = "-".join(date)
    return date

In [181]:
data["date_of_birth"] = data["date_of_birth"].map(unify_date_format)
data["date_of_birth"].head(25)

0     1955-08-04
1     1974-05-17
2     1977-01-13
3     1957-12-28
4     1962-07-07
5     1957-03-16
6     1943-07-04
7     1984-05-13
8     1971-05-10
9     1986-11-13
10    1978-11-10
11    1967-07-20
12    1959-01-25
13    1959-12-05
14    1968-10-05
15    1957-06-28
16    1977-04-02
17    1968-05-22
18    1968-05-22
19    1960-12-25
20    1986-08-30
21    1941-07-25
22    1960-05-01
23    1959-01-25
24    1972-10-31
Name: date_of_birth, dtype: object

Dole môžeme vidieť, že záporný vek vznikol kvôli nesprávnej hodnote dátumu. Na jeho miesto môžeme teda dosadiť
medián veku pacientov s kladným vekom. Týmto odstránime problém záporných hodnôt pri veku.

In [182]:
data.loc[(data["age"] < 0)].date_of_birth

3349    2020-04-22
Name: date_of_birth, dtype: object

In [183]:
age_median = data[(data["age"] > 0)].age.median()
data.loc[(data["age"] < 0), "age"] = int(age_median)

In [184]:
data["age"].describe()

count    3983.000000
mean       51.858147
std        11.768153
min         3.000000
25%        44.500000
50%        52.000000
75%        60.000000
max       113.000000
Name: age, dtype: float64

Čo sa týka atribútu sex, obsahuje len dve hodnoty, ktoré môžeme reprezentovať aj číselne. Je to nutné, keďže v ďalšej
fáze budeme robiť s algoritmami pracujúcimi len s numerickými dátami. Pohlavia budeme reprezentovať nasledovne:
- Male = 1
- Female = 0

In [208]:
data["sex"].unique()

array([' Female', ' Male'], dtype=object)

In [236]:
def fix_sex_value(sex):
    if sex.strip() == "Male":
        return 1
    else:
        return 0

In [237]:
data["sex"] = data["sex"].map(fix_sex_value)
data["sex"].unique()

array([0, 1], dtype=int64)

V minulej časti projektu sme ukázali, že atribút pregnant nadobúda priveľa hodnôt, ktoré symbolizujú ten istý stav.
Preto ich, rovnako ako pohlavie, nahradíme číslami 1 a 0.

In [209]:
print(data["pregnant"].unique())
print(sum(data["pregnant"].isnull()))

['f' 'FALSE' 'T' 'F' 'TRUE' 't' nan]
12


In [271]:
def unify_pregnancy_values(value):

    try:
        if value.strip() in ['t','T','TRUE']:
            return 1
        elif value.strip() in ['f','F','FALSE']:
            return 0
        else:
            return np.nan
        
    except AttributeError:
        return np.nan

In [272]:
data["pregnant"] = data["pregnant"].map(unify_pregnancy_values)
print(data["pregnant"].unique())
print(sum(data["pregnant"].isnull()))

[nan]
3983


Atribút pregnant nastavíme na 0 u všetkých mužov, je možné, že tým odstránime niektoré nulové hodnoty, alebo opravíme
prípadné chyby.

In [279]:
def fix_male_pregnancy(data):
    data.loc[(data.sex == 1), "pregnant"] = 0
    return data

In [280]:
data = fix_male_pregnancy(data)
print(sum(data["pregnant"].isnull()))

12


Počet prázdnych hodnôt ostal rovnaký, no tým sa budeme venovať nižšie, kde vyriešime problémy s prázdnymi hodnotami
všetkých atribútov.

### Odstraňovanie duplicitných záznamov

Nakoniec sme sa rozhodli, že duplicitné záznamy určíme pomocou mena a adresy. Tie následne môžeme odstrániť.
Pre každý záznam v súbore personal_train môžeme priradiť záznam zo súboru other_train, teda celkový výsledný
dataset má mať počet záznamov menšieho z oboch súborov.

In [281]:
print("Number of rows in personal_train.csv ", len(personal_data))
print("Number of rows in other_train.csv ", len(medical_data))
print("Number of rows combined: ", len(data))
print("Number of duplicates: ", len(data[data.duplicated(["name", "address"])]))

Number of rows in personal_train.csv  3933
Number of rows in other_train.csv  3983
Number of rows combined:  3983
Number of duplicates:  50


In [282]:
data = data.drop_duplicates(["name", "address"])
print("Number of rows combined: ", len(data))
print("Number of duplicates: ", len(data[data.duplicated(["name", "address"])]))

Number of rows combined:  3933
Number of duplicates:  0
